In [ ]:
# |default_exp mongo

# DB variables

In [ ]:
# |export

from dotenv import load_dotenv, find_dotenv
from helper import load_env
import os


def get_DB_VARS():
    load_env()
    DB_HOST = os.getenv("DB_HOST")
    DB_NAME = os.getenv("DB_NAME")
    DB_LOGS = os.getenv("DB_LOGS")
    return DB_HOST, DB_NAME, DB_LOGS

# logs

In [ ]:
# |export

from connection import get_collection
from datetime import datetime
import pymongo


def mongolog(operation: str, entite: str, desc: str):
    DB_HOST, DB_NAME, DB_LOGS = get_DB_VARS()
    if DB_LOGS in ["true", "True"]:
        coll_logs = get_collection(DB_HOST, DB_NAME, "logs")
        coll_logs.insert_one(
            {
                "operation": operation,
                "entite": entite,
                "desc": desc,
                "date": datetime.now(),
            }
        )


def print_logs(n: int = 10):
    """
    Print the last n logs
    """
    DB_HOST, DB_NAME, DB_LOGS = get_DB_VARS()
    coll_logs = get_collection(DB_HOST, DB_NAME, "logs")
    for i, log in enumerate(coll_logs.find().sort("date", pymongo.DESCENDING)):
        if i == n:
            break
        print(log)

In [ ]:
print_logs(5)

{'_id': ObjectId('6779a8f8b216345335c98c7e'), 'operation': 'insert', 'entite': 'auteur', 'desc': 'George Sand', 'date': datetime.datetime(2025, 1, 4, 22, 32, 40, 409000)}
{'_id': ObjectId('6779a8f8b216345335c98c7a'), 'operation': 'insert', 'entite': 'auteur', 'desc': 'Jules Verne', 'date': datetime.datetime(2025, 1, 4, 22, 32, 40, 402000)}
{'_id': ObjectId('6779a8f8b216345335c98c76'), 'operation': 'insert', 'entite': 'auteur', 'desc': 'Alexandre Dumas', 'date': datetime.datetime(2025, 1, 4, 22, 32, 40, 394000)}
{'_id': ObjectId('6779a8f8b216345335c98c72'), 'operation': 'insert', 'entite': 'auteur', 'desc': 'Stendhal', 'date': datetime.datetime(2025, 1, 4, 22, 32, 40, 387000)}
{'_id': ObjectId('6779a8f8b216345335c98c6e'), 'operation': 'insert', 'entite': 'auteur', 'desc': 'Albert Camus', 'date': datetime.datetime(2025, 1, 4, 22, 32, 40, 380000)}


# Base entity

c'est une classe de base issue de la refactorisation de Auteur et Editeur proposée par github copilot


> Étapes de Raisonnement pour la Factorisation
> Analyser les Classes Existantes :
> 
> Les classes Auteur et Editeur ont des méthodes similaires pour vérifier l'existence, insérer, mettre à jour et supprimer des documents dans la base de données MongoDB.
> Les méthodes exists, keep, remove, et get_oid sont communes aux deux classes.
> Identifier les Comportements Communs :
> 
> Les comportements communs peuvent être regroupés dans une classe de base.
> Les classes spécifiques (Auteur et Editeur) peuvent hériter de cette classe de base et ajouter des comportements spécifiques si nécessaire.
> Créer une Classe de Base :
> 
> Créer une classe de base BaseEntity qui contient les méthodes communes.
> La classe de base gérera les interactions avec la base de données.
> Hériter de la Classe de Base :
> 
> Les classes Auteur et Editeur hériteront de BaseEntity.
> Elles définiront uniquement les comportements spécifiques à chaque entité.



In [ ]:
# |export

from bson import ObjectId
from connection import get_collection


class BaseEntity:
    def __init__(self, nom: str, collection_name: str):
        """
        BaseEntity is a class that represents a generic entity in the database.
        :param nom: The name of the entity.
        :param collection_name: The name of the collection.
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        self.collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=collection_name
        )
        self.nom = nom

    def exists(self) -> bool:
        """
        Check if the entity exists in the database.
        :return: True if the entity exists, False otherwise.
        """
        return self.collection.find_one({"nom": self.nom}) is not None

    def keep(self):
        """
        Keep the entity in the database.
        """
        if not self.exists():
            mongolog("insert", self.collection.name, self.nom)
            self.collection.insert_one({"nom": self.nom})
        else:
            mongolog("update", self.collection.name, self.nom)

    def remove(self):
        """
        Remove the entity from the database.
        """
        self.collection.delete_one({"nom": self.nom})
        mongolog("delete", self.collection.name, self.nom)

    def get_oid(self) -> ObjectId:
        """
        Get the object id of the entity.
        :return: The object id of the entity. (bson.ObjectId)
        None if does not exist.
        """
        document = self.collection.find_one({"nom": self.nom})
        if document:
            return document["_id"]
        else:
            return None

# Auteur

In [ ]:
# |export


class Auteur(BaseEntity):
    def __init__(self, nom: str):
        """
        Auteur is a class that represents an author in the database auteurs.
        :param nom: The name of the author.
        """
        super().__init__(nom, "auteurs")

In [ ]:
acteur1 = Auteur("Victor Hugo")
acteur1.exists()

True

In [ ]:
acteur1.keep()
acteur1.exists()

True

In [ ]:
acteur1.get_oid()

ObjectId('677945859bf82d43ea3f34c9')

In [ ]:
acteur1.remove()
acteur1.exists()

False

In [ ]:
acteur1.get_oid()

# Editeur

In [ ]:
# |export


class Editeur(BaseEntity):
    def __init__(self, nom: str):
        """
        Editeur is a class that represents a publisher in the database editeurs.
        :param nom: The name of the publisher.
        """
        super().__init__(nom, "editeurs")

In [ ]:
editeur1 = Editeur("Gallimard")
editeur1.exists()

False

In [ ]:
editeur1.keep()
editeur1.exists()

True

In [ ]:
editeur1.get_oid()

ObjectId('677a36762e69c4944df8b0a3')

In [ ]:
editeur1.remove()
editeur1.exists()

False

In [ ]:
editeur1.get_oid()

In [ ]:
print_logs(5)

{'_id': ObjectId('677a36782e69c4944df8b0a5'), 'operation': 'delete', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 1, 5, 8, 36, 24, 956000)}
{'_id': ObjectId('677a36762e69c4944df8b0a2'), 'operation': 'insert', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 1, 5, 8, 36, 22, 509000)}
{'_id': ObjectId('677a36562e69c4944df8b09f'), 'operation': 'delete', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 5, 8, 35, 50, 198000)}
{'_id': ObjectId('677a36482e69c4944df8b09d'), 'operation': 'update', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 5, 8, 35, 36, 332000)}
{'_id': ObjectId('677a33c02e69c4944df8b099'), 'operation': 'delete', 'entite': 'editeur', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 1, 5, 8, 24, 48, 796000)}


# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("mongo helper.ipynb", ".")